## Демо по мотивам статьи 
## Unsupervised Machine Translation Using Monolingual Corpora Only

Основные отличия от алгоритма, описанного в статье:

1. Небольшие различия в архитектуре (GRU вместо LSTM), другой оптимизатор, более простая модель дискриминатора
2. Отстутсвие Attention в Decoder
3. Не добавлял шум в автокодировщик

Скачиваем параллельные предложения из multi30K сразу в предобработанном виде.

Основная предобработка: слова и знаки препинания разделены пробелом, и некоторая работа со спецсимволами (типа '&').

In [1]:
#!wget https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok/train.lc.norm.tok.fr

In [2]:
#!wget https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok/train.lc.norm.tok.en

In [3]:
#!head train.lc.norm.tok.fr

Скачиваем MUSE-вектора, т.е. вектора для слов на разных языках, выровненные таким образом, чтобы косинусное расстояние между схожими словами на разных языках было невелико. 

In [4]:
#!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec#https://s3.amazonaws.com/arrival/embeddings/wiki.multi.en.vec

In [5]:
#!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec#https://s3.amazonaws.com/arrival/embeddings/wiki.multi.fr.vec

Импорт основных библиотек

In [6]:
import io
import numpy as np

import unicodedata
import string
import re
import random
import codecs
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


Из обучалки torch Seq2Seq: удаляем всё, кроме латинских букв и знаков препинания. 
    
Диакритические знаки (черточки над буквами) также удаляем.

In [7]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.strip()

In [8]:
normalizeString(u"bonjour  je suis élève de l'institut'")

'bonjour je suis eleve de l institut'

Функция загрузки векторов MUSE из файла.
Словари векторов содержат много мусорных слов (типа хэштегов), поэтому используем следующую логику:
    
1. Предобрабатываем слова, удаляем нелатинские символы
2. Если после предобработки в словаре встретился дубликат:
        a. Если текущее слово не изменилось после предобработки (т.е. оно скорее всего "хорошее"), то заменяем для данного слова в словаре вектор на текущий
        b. Иначе пропускаем и идем дальше

In [9]:
def load_vec(emb_path):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            orig_word, vect = line.rstrip().split(' ', 1)
            
            word = normalizeString(orig_word)
            vect = np.fromstring(vect, sep=' ')
            if word in word2id:
                print (u'word found twice: {0} ({1})'.format(word, orig_word))
                if orig_word==word:
                    id = word2id[word]
                    vectors[id] = vect
                    print ('rewriting')
                    continue
                else:
                    continue
            vectors.append(vect)
            word2id[word] = len(word2id)
            
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [10]:
en_embedding_tuple = load_vec('./wiki.multi.en.vec')

word found twice:  (-)
word found twice:  (')
word found twice:  ())
word found twice:  (()
word found twice: s (s)
rewriting
word found twice:  (–)
word found twice:  (#)
word found twice:  (%)
word found twice:  (/)
word found twice:  (")
word found twice:  (—)
word found twice:  (}})
word found twice:  (})
word found twice:  ($)
word found twice:  (>)
word found twice:  (+)
word found twice:  (&)
word found twice: www (//www)
word found twice:  (•)
word found twice: p (>p)
word found twice:  (·)
word found twice: en (en)
rewriting
word found twice:  (→)
word found twice:  (%/)
word found twice: com (com/)
word found twice: u (>u)
word found twice:  (£)
word found twice:  (×)
word found twice:  (}}}})
word found twice: d (#d)
word found twice: present (–present)
word found twice:  (}}})
word found twice: or (or%)
word found twice:  (//)
word found twice:  (~)
word found twice:  (°)
word found twice:  (\)
word found twice:  (_)
word found twice:  (⚡)
word found twice: title (_title)
w

word found twice:  (/–)
word found twice: andres (andres)
rewriting
word found twice: debut (début)
word found twice:  (`)
word found twice: link (_link)
word found twice: sao (sao)
rewriting
word found twice: fcc (#fcc)
word found twice:  (►)
word found twice:  (///////)
word found twice:  (¢)
word found twice:  (θ)
word found twice: this (#this)
word found twice: tome (tome)
rewriting
word found twice: etienne (etienne)
rewriting
word found twice: th (th/)
word found twice: b (b+)
word found twice: t (—❤t☮☺☯)
word found twice: b (\b)
word found twice: zu (/zu/)
word found twice: a (/a)
word found twice: data (//data)
word found twice: gfdl (gfdl}})
word found twice: pi (\pi)
word found twice: gamma (\gamma)
word found twice: a (a$)
word found twice:  (τ)
word found twice: th (th–)
word found twice:  (♫)
word found twice: free (_free)
word found twice: film (_film)
word found twice: secure (//secure)
word found twice: college (collège)
word found twice: pyrenees (pyrenees)
rewriting
w

word found twice: blogspot (blogspot\)
word found twice: dona (dona)
rewriting
word found twice: esp (esp}})
word found twice: g (g_)
word found twice: hk (hk$)
word found twice: h (h_)
word found twice: talk (talk%)
word found twice: a (‘a)
word found twice: parent (parent}})
word found twice: german (germán)
word found twice: cite (cité)
word found twice: pa (pā)
word found twice:  (↔)
word found twice:  (ø)
word found twice: marin (marín)
word found twice: cordoba (cordoba)
rewriting
word found twice: south (south%)
word found twice: country (_country)
word found twice: chisinau (chişinău)
word found twice:  (⚔)
word found twice:  (雲)
word found twice: n (n_)
word found twice: tn (/tn/)
word found twice: wikipedia (}}wikipedia)
word found twice: cote (cote)
rewriting
word found twice: zero (_zero)
word found twice: c (c+)
word found twice: gunter (gunter)
rewriting
word found twice: and (—and)
word found twice: city (city_)
word found twice: in (#in)
word found twice: turk (türk)
wo

word found twice: in (īn)
word found twice: list (#list)
word found twice: vladimir (vladimír)
word found twice: and (,and)
word found twice: guarani (guaraní)
word found twice:  (л)
word found twice: if (}#if)
word found twice: uc (đức)
word found twice: ar (är)
word found twice: schaffer (schaffer)
rewriting
word found twice: miro (miro)
rewriting
word found twice: k (k–)
word found twice: article (@article)
word found twice: c (c^)
word found twice: th (th%)
word found twice: summer (summer%)
word found twice: william (#william)
word found twice: women (women%)
word found twice:  (‧)
word found twice: section (&section)
word found twice:  (ο)
word found twice:  (}}}}}}}}}})
word found twice: table (table}})
word found twice:  (русский)
word found twice: bahia (bahía)
word found twice: lim (\lim_)
word found twice: road (road%)
word found twice:  (✈)
word found twice: ibanez (ibanez)
rewriting
word found twice: epsilon (\epsilon)
word found twice:  (%–)
word found twice: aaaaaa (#aaa

word found twice: proposed (/proposed)
word found twice: yucatan (yucatan)
rewriting
word found twice: marques (marqués)
word found twice:  (\})
word found twice:  (српски)
word found twice: saito (saitō)
word found twice: arg (arg}})
word found twice:  (е)
word found twice: mi (/mi²)
word found twice:  (ξ)
word found twice: pre (pré)
word found twice: cedric (cédric)
word found twice: m (m+)
word found twice:  (н)
word found twice: valerie (valérie)
word found twice: michael (michaël)
word found twice: rincon (rincon)
rewriting
word found twice: romania (românia)
word found twice: z (z^)
word found twice:  (%}})
word found twice: xuan (xuân)
word found twice: chah (chāh)
word found twice: talk (tälk)
word found twice: wedge (\wedge)
word found twice: mex (mex}})
word found twice:  (ζ)
word found twice:  (за)
word found twice: daimyo (daimyō)
word found twice: brasov (braşov)
word found twice: ned (ned}})
word found twice: szabo (szabo)
rewriting
word found twice: que (qué)
word found 

...

word found twice: territories (territories%)
word found twice: szymanski (szymański)
word found twice: yi (yì)
word found twice: huang (huáng)
word found twice: conferences (conférences)
word found twice: lwn (//lwn)
word found twice: sf (_sf)
word found twice: avc (avcı)
word found twice: tone (toné)
word found twice: in (_in)
word found twice: sum n (\sum_n)
word found twice:  (╫)
word found twice: age (age%)
word found twice: foreningen (foreningen)
rewriting
word found twice: jimmy (jimmy%)
word found twice: alan (alan%)
word found twice: munter (münter)
word found twice: white (white,)
word found twice: edit (#edit)
word found twice: quintin (quintín)
word found twice: creek (creek%)
word found twice: org about (org/about/)
word found twice: seances (seances)
rewriting
word found twice: palmares (palmarès)
word found twice: hang (hàng)
word found twice: r (r%)
word found twice: ace (#ace)
word found twice:  (войны)
word found twice: aetius (aëtius)
word found twice: kastel (kaštel

word found twice: dos (dos/)
word found twice: asatru (asatru)
rewriting
word found twice:  ()
word found twice: meroe (meroë)
word found twice: fric (fric)
rewriting
word found twice: baena (baena}})
word found twice: nimes (nimes)
rewriting
word found twice: cuauhtemoc (cuauhtemoc)
rewriting
word found twice: zp (žp)
word found twice: estee (estee)
rewriting
word found twice: name (name%)
word found twice: democratico (democratico)
rewriting
word found twice: nabla (nabla)
rewriting
word found twice: esteve (estève)
word found twice: assembly (assembly%)
word found twice:  (♯)
word found twice: la (‘la)
word found twice: vukovic (vukovic)
rewriting
word found twice: v (və)
word found twice: grand (#grand)
word found twice: makhachkala (makhachkala}})
word found twice: m m (m/m)
word found twice: theater (&theater)
word found twice: abi (abi%)
word found twice: cacic (čačić)
word found twice: h (δh)
word found twice:  (остров)
word found twice: tunisia (tunisia}})
word found twice:  

word found twice: park (park/)
word found twice: sea (sea%)
word found twice: seppala (seppälä)
word found twice: pomte (pomte)
rewriting
word found twice: queensland (queensland%)
word found twice: pera (pêra)
word found twice: lake (lake}})
word found twice: kien (kiến)
word found twice:  (меня)
word found twice: companies (companies%)
word found twice: com world (com/world/)
word found twice: mari (marí)
word found twice: cerebral (cerebral%)
word found twice: leonce (leonce)
rewriting
word found twice: z (+z)
word found twice: shir (shīr)
word found twice: dubs (dübs)
word found twice: rosler (rosler)
rewriting
word found twice:  (­)
word found twice: r (r_^)
word found twice: four (four%)
word found twice: beta (\beta^)
word found twice:  (/}}})
word found twice: puerto (puerto%)
word found twice: openlibrary (openlibrary)
rewriting
word found twice: arabia (arabia}})
word found twice: sanger (sånger)
word found twice: f (–f)
word found twice:  (☮ღ☺)
word found twice: papers (/pap

word found twice: suma (šuma)
word found twice: hms (#hms)
word found twice: ibrahim (ibrāhīm)
word found twice: sr ($sr)
word found twice: maya (māyā)
word found twice: cf (cf}})
word found twice: mahmudabad (maḩmūdābād)
word found twice:  (علی)
word found twice: charter (charter%)
word found twice: base (base%)
word found twice: alt (⢀alt)
word found twice: mole (molé)
word found twice: yuta (yūta)
word found twice: detail (detail/)
word found twice: jauregui (jauregui)
rewriting
word found twice: israelite (israélite)
word found twice: paramo (paramo)
rewriting
word found twice: ukic (ukić)
word found twice: has (—has)
word found twice: hopital (hopital)
rewriting
word found twice: huskies (huskies%)
word found twice: kosei (kōsei)
word found twice: be (\be)
word found twice:  (⢌)
word found twice: i (•i)
word found twice: miles (miles/)
word found twice:  (ל)
word found twice:  (связей)
word found twice:  (\^)
word found twice: reves (reves)
rewriting
word found twice:  (__________

word found twice: go (go%)
word found twice: cyclone (#cyclone)
word found twice: mw (~mw)
word found twice:  (⨹)
word found twice: judetul (judeţul)
word found twice: despotovic (despotovic)
rewriting
word found twice: japan (japan%)
word found twice: against (against%)
word found twice: one (one,)
word found twice: friedrich (#friedrich)
word found twice: gorecki (gorecki)
rewriting
word found twice: eleonore (éléonore)
word found twice:  (}},}})
word found twice: alt (⣱alt)
word found twice: panic (panić)
word found twice: softball (softball%)
word found twice:  (ما)
word found twice: asp (asp/)
word found twice: proteges (proteges)
rewriting
word found twice: april june (april→june)
word found twice: long (lóng)
word found twice: ve (ﬁve)
word found twice: yuto (yūto)
word found twice: lak (lắk)
word found twice: eta (\eta^)
word found twice: katarina (katarína)
word found twice: artistico (artístico)
word found twice:  (╱)
word found twice: etc (/etc)
word found twice: i (ì)
word 

In [11]:
fr_embedding_tuple = load_vec('./wiki.multi.fr.vec')

word found twice:  (')
word found twice:  (-)
word found twice:  ())
word found twice:  (()
word found twice: a (a)
rewriting
word found twice: s (s)
rewriting
word found twice:  (»)
word found twice:  (}})
word found twice:  (#)
word found twice:  (/)
word found twice: ou (où)
word found twice:  (%)
word found twice:  (–)
word found twice: ne (né)
word found twice:  (│)
word found twice:  (+)
word found twice:  (•)
word found twice:  (—)
word found twice: des (dès)
word found twice:  (})
word found twice:  (·)
word found twice:  (&)
word found twice: la (là)
word found twice: annee (&annee)
word found twice:  (°)
word found twice: comte (comte)
rewriting
word found twice:  (†)
word found twice: donne (donné)
word found twice: cote (côte)
word found twice: n (n°)
word found twice:  ( )
word found twice:  (>)
word found twice: demande (demandé)
word found twice: passe (passé)
word found twice: wikipedia (wikipedia)
rewriting
word found twice: situe (situe)
rewriting
word found twice: ut

word found twice: capture (capturé)
word found twice:  (⇒)
word found twice: marches (marches)
rewriting
word found twice: right (\right)
word found twice: prete (prêté)
word found twice: prefere (préféré)
word found twice: fur (fur)
rewriting
word found twice: chaine (chaine)
rewriting
word found twice: reference (reference)
rewriting
word found twice: parait (parait)
rewriting
word found twice: associe (associe)
rewriting
word found twice: separe (séparé)
word found twice: passes (passés)
word found twice: ile (ile)
rewriting
word found twice: hotel (hotel)
rewriting
word found twice: entraine (entraîné)
word found twice: edit (édit)
word found twice: television (television)
rewriting
word found twice: formes (formés)
word found twice: apporte (apporté)
word found twice: provoque (provoqué)
word found twice: regiment (regiment)
rewriting
word found twice: the (thé)
word found twice: confie (confié)
word found twice: accuse (accuse)
rewriting
word found twice: felix (felix)
rewriting


word found twice: region (region)
rewriting
word found twice: limites (limités)
word found twice: rentre (rentré)
word found twice: affecte (affecte)
rewriting
word found twice: atteste (atteste)
rewriting
word found twice: creuse (creusé)
word found twice: maitre (maitre)
rewriting
word found twice: forge (forgé)
word found twice: x (x^)
word found twice: adapte (adapte)
rewriting
word found twice: fixes (fixés)
word found twice: ain (aïn)
word found twice: m (m²)
word found twice: refugie (réfugié)
word found twice: justifie (justifié)
word found twice: ramene (ramené)
word found twice: eut (eût)
word found twice: illustres (illustrés)
word found twice: sao (sao)
rewriting
word found twice: mat (mât)
word found twice: borde (borde)
rewriting
word found twice: plante (planté)
word found twice: sanchez (sanchez)
rewriting
word found twice: jerome (jerome)
rewriting
word found twice: barthelemy (barthélémy)
word found twice: opere (opéré)
word found twice: different (diffèrent)
word fou

word found twice: rassemble (rassemblé)
word found twice: attaques (attaqués)
word found twice: colle (collé)
word found twice: frederic (frederic)
rewriting
word found twice: q (q}})
word found twice: figure (figuré)
word found twice: ecarte (écarte)
word found twice:  (и)
word found twice: menaces (menacés)
word found twice: oscar (óscar)
word found twice: boheme (bohème)
word found twice: reproche (reproché)
word found twice: regional (regional)
rewriting
word found twice: teste (teste)
rewriting
word found twice: leningrad (leningrad)
rewriting
word found twice: mele (mêlé)
word found twice: rose (rosé)
word found twice: lambda (\lambda)
word found twice: vicomte (vicomté)
word found twice: consulte (consulte)
rewriting
word found twice: decline (décliné)
word found twice: quebec (quebec)
rewriting
word found twice: equipe (equipe)
rewriting
word found twice: experience (experience)
rewriting
word found twice: tome (tomé)
word found twice: taiwan (taiwan)
rewriting
word found twice

word found twice: avila (avila)
rewriting
word found twice: accelere (accéléré)
word found twice: regroupe (regroupé)
word found twice: d (%d)
word found twice: sigma (\sigma)
word found twice: perce (perce)
rewriting
word found twice: jozef (józef)
word found twice: approche (approché)
word found twice: hat (\hat)
word found twice: piege (piégé)
word found twice: incite (incité)
word found twice: julian (julián)
word found twice: fouille (fouillé)
word found twice: negocie (négocié)
word found twice: emilie (emilie)
rewriting
word found twice: evite (évité)
word found twice: epargne (épargné)
word found twice: internes (internés)
word found twice: generalise (généralise)
word found twice: napoleon (napoleon)
rewriting
word found twice: m (m_)
word found twice: angelique (angelique)
rewriting
word found twice: vertebres (vertèbres)
word found twice: energie (energie)
rewriting
word found twice: henin (henin)
rewriting
word found twice: durer (dürer)
word found twice: explore (exploré)


...

word found twice: emilienne (emilienne)
rewriting
word found twice: france (france/)
word found twice:  (の)
word found twice:  (⎕)
word found twice: gabriele (gabrièle)
word found twice: and (and%)
word found twice: honorio (honorio)
rewriting
word found twice: solea (soléa)
word found twice: mc (mc^)
word found twice: rocio (rocio)
rewriting
word found twice: systemes (systemes)
rewriting
word found twice: lh (lh/)
word found twice: azema (azema)
rewriting
word found twice: so (so_)
word found twice: predecesseur (prédecesseur)
word found twice: caitlin (caitlín)
word found twice: etudes ( études)
word found twice: babord (babord)
rewriting
word found twice: emboite (emboîté)
word found twice: red (_red)
word found twice: critique (critique »)
word found twice: goju (gōjū)
word found twice:  (искусство)
word found twice: reactualise (réactualise)
word found twice:  (⊆)
word found twice: bottes (bottés)
word found twice: reglement (reglement)
rewriting
word found twice: modifier ( modi

word found twice: u (ŭ)
word found twice: cage (cagé)
word found twice: annote (annote)
rewriting
word found twice: beau ( beau)
word found twice: melin (mélin)
word found twice: bandai (bandaï)
word found twice: petrov (pétrov)
word found twice: pole ( pôle)
word found twice: c (ć)
word found twice: chronique ( chronique)
word found twice: beche (bèche)
word found twice: leur ( leur)
word found twice: vivre ( vivre)
word found twice: interprete (interprête)
word found twice: commune (commune}})
word found twice: politiques (politiques »)
word found twice: jeunes (jeunes »)
word found twice: biogeographie (biogeographie)
rewriting
word found twice: espagne (espagne »)
word found twice: pospisil (pospíšil)
word found twice: nystrom (nystrom)
rewriting
word found twice: surtaxe (surtaxé)
word found twice: amin (amîn)
word found twice: menotte (menotte)
rewriting
word found twice: georgien (georgien)
rewriting
word found twice: remora (rémora)
word found twice: hegesippe (hégesippe)
word 

word found twice:  (兔)
word found twice: theodor (théodor)
word found twice: plutot ( plutôt)
word found twice: luge (lüge)
word found twice: defriche (défriche)
word found twice: abl (%abl)
word found twice: moreri (moreri)
rewriting
word found twice: volte (volté)
word found twice: e f (e,f)
word found twice: eons (eons)
rewriting
word found twice: juillet ( juillet)
word found twice: phenyl (phenyl)
rewriting
word found twice: jehan (jéhan)
word found twice: izia (izïa)
word found twice: terres ( terres)
word found twice: ngu (ngữ)
word found twice: derive (derive)
rewriting
word found twice: ici (içi)
word found twice:  ( }})
word found twice: selecao (seleçao)
word found twice: body (_body)
word found twice: allaite (allaité)
word found twice: o o (ʻōʻō)
word found twice: ade (adé)
word found twice: la (la♭)
word found twice: croix (croix »)
word found twice: balalaika (balalaika)
rewriting
word found twice: non (non…)
word found twice: codes (/codes)
word found twice: walder (wäl

word found twice: score (score_)
word found twice: publics (publics,)
word found twice: remond (remond)
rewriting
word found twice: tiberius (tibérius)
word found twice: borbon (borbon)
rewriting
word found twice: aerienne (aerienne)
rewriting
word found twice: vaisse (vaisse)
rewriting
word found twice: heaulme (heaulmé)
word found twice: championship (championship}})
word found twice: kato (katô)
word found twice: dax (_dax)
word found twice: reville (reville)
rewriting
word found twice: attente (attenté)
word found twice:  (алексей)
word found twice: catche (catché)
word found twice: mela (méla)
word found twice: h (_h)
word found twice: futs (futs)
rewriting
word found twice: replicant (replicant)
rewriting
word found twice: nuit (nuit…)
word found twice: bedouin (bedouin)
rewriting
word found twice: cemea (cemea)
rewriting
word found twice: valle (vallé)
word found twice: annee (année )
word found twice: navratil (navrátil)
word found twice: n (+n)
word found twice: nevroses (névr

word found twice: dechaine (déchaine)
word found twice: junjo (junjō)
word found twice: universita (universita)
rewriting
word found twice: quebec (québec}})
word found twice: g (ĝ)
word found twice: an (ân)
word found twice: championnat (championnat}})
word found twice: ragout (ragout)
rewriting
word found twice: peaufine (peaufiné)
word found twice: guei (gueï)
word found twice: exiguite (exigüité)
word found twice: burgi (bürgi)
word found twice: p i (p_i^)
word found twice: borras (borrás)
word found twice: appel ( appel)
word found twice: ecume (écumé)
word found twice: qu (#qu)
word found twice: civica (cívica)
word found twice: mysterio (mystério)
word found twice: menaiel (ménaïel)
word found twice: moe (moé)
word found twice: reve (rève)
word found twice: kobe (kôbe)
word found twice: makeieff (makeieff)
rewriting
word found twice: rebuffat (rebuffat)
rewriting
word found twice: kiraly (kiraly)
rewriting
word found twice: lowy (lowy)
rewriting
word found twice: samtskhe (samts

Функция из MUSE: находим ближайшие вектора из другого языка. 

Посмотрим, насколько хорошо работает выравнивание.

In [12]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [13]:
for word in ['cat','dog','human','student','computer']:
    get_nn(word, en_embedding_tuple[0], en_embedding_tuple[1], fr_embedding_tuple[0], fr_embedding_tuple[1], K=3)

Nearest neighbors of "cat":
0.6131 - chat
0.5779 - cat
0.5504 - chien
Nearest neighbors of "dog":
0.7998 - chien
0.7052 - chiens
0.6259 - chienne
Nearest neighbors of "human":
0.7290 - humain
0.7280 - humains
0.6391 - humaine
Nearest neighbors of "student":
0.7106 - etudiante
0.6421 - etudiantes
0.5901 - professeurs
Nearest neighbors of "computer":
0.7823 - informatique
0.7696 - ordinateur
0.7398 - ordinateurs


Класс Lang отвечает за обработку языка.

SOS_token --- идентификатор начала предложения.

EOS_token --- идентификатор конца предложения.

In [14]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name, embedding_tuple):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = [ "SOS", "EOS"]
        self.embedding_tuple = embedding_tuple    
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:   
            if word in self.embedding_tuple[2]:
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word.append(word)
                self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def get_matrix(self):
        """
        Получаем матрицу слово -> вектор для всех слов, которые встретились в тексте.
        Вектор для начала предложения заменяем нулевым, 
        для конца предложения --- единичным (можно заменить на случайный вектор).
        """
        dim = self.embedding_tuple[0].shape[1]
        matrix = np.zeros((self.n_words, dim))        
        matrix[0] = np.zeros(dim)
        matrix[1] = np.ones(dim)
        for id, word in enumerate(self.index2word[2:]):
            id = id+2
            word_id = self.embedding_tuple[2][word]
            vector = self.embedding_tuple[0][word_id]
            matrix[id] = vector
        return matrix

In [15]:
# Предложение представляется как набор идентификаторов слов. 
# Чтобы поместить несколько предложений в одну матрицу, нужно дополнить каждое предложение токенами конца предложения.
def pad_seq(seq, length):
    
    seq += [EOS_token for i in range(length - len(seq))]
    return seq

Функции-утилиты для считывания текста.

В дальнейшем ограничимся только предложениями длины <= 10 для экономии памяти.

In [16]:
def readLangs(lang1, lang2, emb1, emb2,  prefix):
    print("Reading lines...")

    # Read the file and split into lines
    lines1 = codecs.open(prefix+lang1, encoding='utf-8').\
        read().strip().split('\n')
    lines2 = codecs.open(prefix+lang2, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    lines1 = [normalizeString(s) for s in lines1]
    lines2 = [normalizeString(s) for s in lines2]
   
    input_lang = Lang(lang1, emb1)
    output_lang = Lang(lang2, emb2)

    return input_lang, output_lang, lines1, lines2

In [17]:
MAX_LENGTH = 10


def filter_line(line):
    return len(line.split(' ')) < MAX_LENGTH
    
def filter_lines(lines):
    return [line for line in lines if filter_line(line)]

In [18]:
def prepareData(lang1, lang2, emb1, emb2,  prefix):
    """
    Возвращает два объекта-языка, предложения для обучения
    и пары параллельных предложений для промежуточной валидации результата
    """
    input_lang, output_lang, lines1, lines2 = readLangs(lang1, lang2,  emb1, emb2, prefix)
    print("Read %s sentence pairs" % len(lines1))
    pairs = [(l1, l2) for l1, l2 in zip(lines1, lines2) if filter_line(l1) and filter_line(l2)]
    
    # по условиями эксперимента у нас нет параллельных предложений. Для чистоты эксперимента  перемешаем их.
    np.random.shuffle(lines1)
    np.random.shuffle(lines2)

    lines1 = filter_lines(lines1)
    lines2 = filter_lines(lines2)
    
    min_lines = min(len(lines1), len(lines2))
    lines1, lines2 = lines1[:min_lines], lines2[:min_lines]
    
    print("Trimmed to %s sentence pairs" % min_lines)
    print("Counting words...")
    for l1, l2 in zip(lines1, lines2):
        input_lang.addSentence(l1)
        output_lang.addSentence(l2)
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, lines1, lines2,  pairs


input_lang, output_lang, lines1, lines2, pairs = prepareData('fr', 'en', fr_embedding_tuple, en_embedding_tuple, 
                                             'train.lc.norm.tok.')
# в качестве щашумленного перевода предложений пока просто возьмем непараллельные пары
tr_lines1,tr_lines2 = lines2[:], lines1[:]

Reading lines...
Read 29000 sentence pairs
Trimmed to 3510 sentence pairs
Counting words...
Counted words:
fr 2794
en 2826


Функции кодирования предложений в последовательность идентификаторов слов

In [19]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


# функция возвращает матрицы случайных предложений и их зашумленных версий в pytorch-формате.
# batch_size --- количество предложений.
def random_batch(batch_size):
    input_seqs = []
    target_seqs = []
    tr_input_seqs = []
    tr_target_seqs = []
    
    # Choose random pairs
    for i in range(batch_size):
        id1 = random.choice(range(len(lines1)))
        id2 = random.choice(range(len(lines2)))
        line1 = lines1[id1]
        line2 = lines2[id2]
        tr_line1 = tr_lines1[id1]
        tr_line2 = tr_lines2[id2]
        
        
        input_seqs.append(indexesFromSentence(input_lang, line1))
        target_seqs.append(indexesFromSentence(output_lang, line2))
        
        tr_input_seqs.append(indexesFromSentence(output_lang, tr_line1))
        tr_target_seqs.append(indexesFromSentence(input_lang, tr_line2))
        
        
    input_length = max([len(s) for s in input_seqs])
    target_length = max([len(s) for s in target_seqs])
    tr_input_length = max([len(s) for s in tr_input_seqs])
    tr_target_length = max([len(s) for s in tr_target_seqs])
    
    
    # For input and target sequences, get array of lengths and pad with 0s to max length    
    input_padded = [pad_seq(s, input_length) for s in input_seqs]    
    target_padded = [pad_seq(s, target_length) for s in target_seqs]
    
    tr_input_padded = [pad_seq(s, tr_input_length) for s in tr_input_seqs]    
    tr_target_padded = [pad_seq(s, tr_target_length) for s in tr_target_seqs]

    
    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = torch.tensor(input_padded, dtype=torch.long,device=device).transpose(0, 1)
    target_var = torch.tensor(target_padded, dtype=torch.long,device=device).transpose(0, 1)
    tr_input_var = torch.tensor(tr_input_padded, dtype=torch.long,device=device).transpose(0, 1)
    tr_target_var = torch.tensor(tr_target_padded, dtype=torch.long,device=device).transpose(0, 1)
    
    
    return input_var, target_var, tr_input_var, tr_target_var

In [20]:
batch = random_batch(30)

Посмотрим матрицы для слов из каждого языка для использования в Seq2Seq

In [21]:
fr_matrix = torch.FloatTensor(input_lang.get_matrix())


In [22]:
en_matrix = torch.FloatTensor(output_lang.get_matrix())


Простой класс однослойной нейросети с 1000 нейронов на скрытом слое, будет использоваться как дискриминатор.

Входная размерность --- 300, совпадает с размерностью слов и размерностью скрытого пространства Seq2Seq,

In [23]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(300,1000)
        self.fc2 = nn.Linear(1000, 1)
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        y = torch.nn.functional.sigmoid(self.fc2(x))
        
        return y

класс Encoder.

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, matrix, gru = None):
        """
        gru -- если не None, берет готовую модель gru и использует для своего языка.
        Параметр требуется, чтобы использовать одну и ту же GRU-модель для двух энкодеров с разных языков.
        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        
        self.embedding = nn.Embedding.from_pretrained(matrix)
        self.embedding.requires_grad = False
        
        if not gru:
            self.gru = nn.GRU(hidden_size, hidden_size)
        else:
            self.gru = gru
            

    def forward(self, input, batch_size, hidden):
        embedded = self.embedding(input).view(1, batch_size, self.hidden_size)
        output = embedded
        
        output, hidden = self.gru(output, hidden)
        return output, hidden
    

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

Decoder. 

Важно: в отличие от статьи, я не использовал Attention. 

Attention можно взять из официальной обучалки pytorch seq2seq, но он там работает с одним предложением за одну операцию.
Лучше погуглить "seq2seq pytorch batch"

In [25]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, matrix, gru = None):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(matrix)
        self.embedding.requires_grad = False
        if not gru:
            self.gru = nn.GRU(hidden_size, hidden_size)
        else:
            self.gru = gru
            
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, batch_size):
        output = self.embedding(input).view(1, batch_size, self.hidden_size)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

Функции для проведения одной итерации оптимизации.

teacher_forcing_ratio отвечает вероятность, что в качестве слов при декоде мы будем получать подсказку, а не будем использовать те слова, что были раскодированы декодером самостоятельно на предыдущих шагах. 

In [26]:
teacher_forcing_ratio = 0.5

def encode_decode(input_tensor,  encoder, decoder, target_tensor=None):
    # раскодирует и декодирует.
    # input_tensor, target_tensor --- матрицы размером
    #    <Количество предложений в батче> * <Максимальная длина предложения в батче>
    # возвращает последовательность идентификаторов слов от декодера и скрытый вектор от энкодера
    batch_size = input_tensor.size(1)
    encoder_hidden = encoder.initHidden(batch_size)
    
    input_length = input_tensor.size(0)
    if target_tensor is None:
        target_tensor = input_tensor
    
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(input_length, batch_size, encoder.hidden_size, device=device)

    for ei in range(input_length):
    
        encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]
    
    
    
    decoder_input = torch.tensor([[SOS_token]*batch_size], device=device)
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    outputs = []
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, batch_size)        
        outputs.append(decoder_output)        
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]  # Teacher forcing            
        else:
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            #if decoder_input == EOS_token:
            #    break
    return outputs,  encoder_hidden


def train(source_tensor, target_tensor, translated_source_tensor, translated_target_tensor, 
          encoder_source, encoder_target,
          decoder_source, decoder_target, discriminator,  
          optimizer,   discriminator_optimizer, 
          criterion, cross_entropy, ae_coef = 1.0, translate_coef = 0.0, disc_coef = 1.0,   max_length=MAX_LENGTH):
    """
    Одна итерация оптимизации
    source_tensor --- матрица предложений первого языка
    target_tensor --- матрица предложений второго языка
    translated_source_tensor --- зашумленный перевод первого языка
    translated_target_tensor --- зашумленный перевод второго языка
    encoder_target, decoder_source, encoder_target, decoder_target --- seq2seq кодировщики
    discriminator --- дискриминаторная сеть
    optimizer, discriminator_optimizer --- оптимизаторы Seq2Seq и дискриминатора
    criterion --- функция ошибки для Seq2Seq. Принимает на вход предложения и матрицу ответа от декодера
    cross_entropy --- функция ошибки для дискриминатора
    """
    
    batch_size = source_tensor.size(1)
    
    # автокодировщики
    source_to_source, source_hidden = encode_decode(source_tensor, encoder_source, decoder_source )
    target_to_target, target_hidden = encode_decode(target_tensor, encoder_target, decoder_target)
    
    # зануляем градиент от ошибки Seq2Seq
    optimizer.zero_grad()
    
    
    loss = 0
    
    # считаем ошибку от первого автокодировщика
    for di in range(len(source_to_source)):
        decoder_output = source_to_source[di]
        loss += criterion(decoder_output, source_tensor[di])*ae_coef
    
    
    #считаем ошибку от второго автокодировщика
    for di in range(len(target_to_target)):
        decoder_output = target_to_target[di]
        loss += criterion(decoder_output, target_tensor[di])*ae_coef
    
    # будем считать, что объекты из ПЕРВОГО языка принадлежат  классу "1", 
    # объекты ВТОРОГО языка --- классу "0"
    classes = torch.tensor(np.array([1.0]*batch_size + [0.0]*batch_size), dtype=torch.float, device=device)
    
    # Смотрим, что предсказал дискриминатор
    source_hidden_predict = discriminator(torch.stack([target_hidden, source_hidden]))
    # На данном этапе мы хотим обмануть дискриминатор, поэтому будем минимизировать долю правильных ответов,
    # т.е. минимизировать ошибку между ответами дискриминатора и НЕПРАВИЛЬНЫМИ классами.
    loss += cross_entropy(source_hidden_predict, classes)*disc_coef
    
    # аналогично автокодирощикам, считаем ошибку на зашумленном переводе
    if translate_coef > 0.0:
        translated_source_to_source, _ = encode_decode(translated_source_tensor, encoder_target, decoder_source, target_tensor=source_tensor)
        translated_target_to_target, _ = encode_decode(translated_target_tensor, encoder_source, decoder_target, target_tensor=target_tensor)
        
        for di in range(min(len(translated_source_to_source), len(source_tensor))):
            decoder_output = translated_source_to_source[di]
            loss += criterion(decoder_output, source_tensor[di])*ae_coef

        for di in range(min(len(translated_target_to_target), len(target_tensor))):
            decoder_output = translated_target_to_target[di]
            loss += criterion(decoder_output, target_tensor[di])*ae_coef



    # подсчет градиента от ошибки
    loss.backward()

    # запуск оптимизации в сторону антиградиента
    optimizer.step()
    
    # информация для отладки
    avg_len = (len(target_tensor)+len(source_tensor))/2
    
    # обнуляем градиент и ошибку для дискриминатора
    d_loss = 0     
    discriminator_optimizer.zero_grad()
    
    _, source_hidden = encode_decode(source_tensor, encoder_source, decoder_source)
    _, target_hidden = encode_decode(target_tensor, encoder_target, decoder_target)
    
    
    # теперь минимизируем ошибку между предсказанным и правильным классами
    source_hidden_predict = discriminator(torch.stack([source_hidden, target_hidden]))    
    d_loss += cross_entropy(source_hidden_predict, classes)

    d_loss.backward()

    discriminator_optimizer.step()
    
              
    
    return loss.item() / avg_len, d_loss.item()

In [27]:
import time

def trainIters(encoder_source,encoder_target,  decoder_source,  decoder_target,discriminator, 
               n_iters, print_every=1000,  learning_rate=0.001):
    # глобальная процедура оптимизации 
    
    
    global tr_lines1, tr_lines2
    
    optimizer = optim.Adam(list(encoder_source.parameters()) +  list(decoder_source.parameters())+\
                            list(encoder_target.parameters())+  list(decoder_target.parameters()),
                           lr=learning_rate)
    
    optimizer2 = optim.Adam(discriminator.parameters(),
                           lr=learning_rate)
    
    
    criterion = nn.NLLLoss()
    criterion2 = nn.BCELoss()
    print_loss_total = [0, 0]

    for iter in range(1, n_iters + 1):    
        batch = random_batch(25)
        
        # уровень доверия к зашумленному переводу будет увеличиваться в процессе оптимизации. 
        # поскольку в первое время мы не обладаем никаким переводом, то изначально коэффициент будет нулевым.
        if iter < print_every:
            tr_coef = 0.0
        else:
            tr_coef = 1.0/n_iters
        loss = train(batch[0], batch[1], batch[2], batch[3],  encoder_source,encoder_target,  decoder_source,  decoder_target,
                     discriminator, optimizer, optimizer2, criterion, criterion2, translate_coef = tr_coef)
        print_loss_total[0] += loss[0]
        print_loss_total[1] += loss[1]
        

        if iter % print_every == 0:
            # выводи примеры перевода, среднюю ошибку и делаем новый шумный перевод, 
            # каждую 'print_every' итерацию.
            print_loss_avg0 = print_loss_total[0] / print_every
            print_loss_avg1 = print_loss_total[1] / print_every
            print_loss_total = [0,0]
            print (iter, print_loss_avg0, print_loss_avg1)
            print ('_'*10)
            evaluateRandomly(encoder_source, decoder_target,n=3)
            print ('_'*10)
            evaluateRandomly(encoder_source, decoder_source,langid=0, n=3)
            print ('_'*10)
            evaluateRandomly(encoder_target, decoder_target, langid=1, n=3)
            
            tr_lines1, tr_lines2 = make_translation()

In [28]:
def make_translation():
    """
    Построение зашуменного перевода. 
    Можно существенно ускорить, если переводить батчем.
    """
    max_length = MAX_LENGTH
    tr_lines1, tr_lines2 = [],[]
    id = 0
    for line in lines1:
        id+=1
        if id%1000 == 0:
            print ('translating source', id)
        input_tensor = tensorFromSentence(input_lang, line)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder_source.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder_source.hidden_size, device=device)

        
        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder_source(input_tensor[ei],1,
                                                     encoder_hidden)
            
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder_target(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:                
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        tr_lines1.append(u' '.join(decoded_words))
    
    # можно устранить дублирование кода, не дошли руки, прим. Олег
    id  = 0 
    for line in lines2:
        id+=1
        if id%1000 == 0:
            print ('translation target', id)
        input_tensor = tensorFromSentence(output_lang, line)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder_target.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder_target.hidden_size, device=device)

        
        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder_target(input_tensor[ei],1,
                                                     encoder_hidden)
            
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder_source(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                
                break
            else:
                decoded_words.append(input_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        tr_lines2.append(u' '.join(decoded_words))
        
    
    return tr_lines1, tr_lines2
#tr_lines1, tr_lines2 = make_translation()

In [29]:
def evaluate(encoder, decoder, sentence, encoded_lang,  decoder_lang, max_length=MAX_LENGTH):
    """
    Процедура промежуточной валидации перевода
    """
    with torch.no_grad():
        input_tensor = tensorFromSentence(encoded_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(1)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(min(MAX_LENGTH, input_length)):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],1,
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, 1)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(decoder_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
    
def evaluateRandomly(encoder, decoder, langid = -1,  n=10):
    """
    Берем n предложений и смотрим качество на них.
    Если langid == -1 --- смотрим качество перевода.
    Если langid == 0 или == 1, смотирм качество восстановления автокодировщиком.
    """
    for i in range(n):
        if langid == -1:
            id0 = 0
            id1 = 1
            enc_lang= input_lang
            dec_lang = output_lang
        else:            
            id0 = langid
            if langid == 0:
                enc_lang = input_lang
                dec_lang = input_lang
            else:
                enc_lang = output_lang
                dec_lang = output_lang
        if langid==-1:
                
            pair = random.choice(pairs)
        else:
            pair = [random.choice(lines1), random.choice(lines2)]
        print   ('>', pair[id0])
        if langid==-1:
            print ('=', pair[id1])
        
            
        output_words = evaluate(encoder, decoder, pair[id0], enc_lang, dec_lang)
        output_sentence = ' '.join(output_words)
        print ('<', output_sentence, '\n')
        
    

Создание сетей и запуск обучения.

Для информации ниже: пример вывода валидации на последних 10000 итераций.

Видно, что перевод работает, хотя и не всегда корректно.

Зато почти идеально выполняется восстановление предложений.

Добавление шума в автокодировщик (как в статье) исправит положение.

In [30]:

hidden_size = 300

encoder_source = EncoderRNN(input_lang.n_words, hidden_size, fr_matrix).to(device)
encoder_target = EncoderRNN(output_lang.n_words, hidden_size, en_matrix, gru = encoder_source.gru).to(device)

decoder_source = DecoderRNN(hidden_size, input_lang.n_words, fr_matrix).to(device)
decoder_target = DecoderRNN(hidden_size, output_lang.n_words, en_matrix, gru = decoder_source.gru).to(device)

#disc = Net1()
#disc.cuda()


In [32]:
disc = Net1()
disc.cuda()
trainIters(encoder_source,encoder_target,  decoder_source,  decoder_target, disc, 100000, print_every=5000)

AssertionError: Torch not compiled with CUDA enabled

Сохраняем модели

In [ ]:
torch.save(encoder_source.gru.state_dict(), 'monolingual_seq2seq_fr_en_enc')
torch.save(decoder_source.gru.state_dict(), 'monolingual_seq2seq_fr_en_dec')